### CryptoMood Initial Version Steps

1. **Twitter**
   - [ ] Access the Twitter API.
   - [ ] Get tweets by filtering with crypto-related hashtags.
   - [ ] Handle API rate limits and implement pagination.
   - [ ] Preprocess tweets to remove noise (URLs, mentions, etc.).

2. **Reddit**
   - [ ] Access the Reddit API.
   - [ ] Get posts from crypto-focused subreddits.
   - [ ] Handle API rate limits and implement pagination.
   - [ ] Preprocess posts to improve sentiment analysis accuracy.

3. **Sentiment Analysis**
   - [ ] Use the TextBlob sentiment analysis tool.
   - [ ] Analyze sentiment of tweets and posts.
   - [ ] Extend VADER with crypto-specific lexicon for better accuracy.

4. **Data Visualization**
   - [ ] Create interactive dashboards with Plotly/Dash.
   - [ ] Display sentiment trends over time.
   - [ ] Visualize sentiment distribution among different cryptocurrencies.

5. **Access Coinbase API**
   - [ ] Fetch current prices of top 10 cryptocurrencies identified by sentiment analysis.
   - [ ] Compare sentiment analysis results with historical price data.

### Additional Enhancements

- [ ] Implement robust error handling for API interactions.
- [ ] Consider data privacy and ethical aspects when presenting user-generated content.
- [ ] Plan for scalability and transitioning from a notebook to a more user-friendly application.
- [ ] Explore incorporating additional data sources like news articles in future versions.
- [ ] Investigate custom machine learning models tailored to cryptocurrency discussions.


## Sentiment Analysis Tool

In [4]:
# Using Textblob to analyze sentiment example
from textblob import TextBlob

text = "Bitcoin is having a great day!"
blob = TextBlob(text)
sentiment = blob.sentiment

print(f"Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}")


# Polarity is a float within the range [-1.0, 1.0], where -1 means negative sentiment and 1 means positive sentiment.
# Subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.


Polarity: 1.0, Subjectivity: 0.75


## Twitter API

In [12]:
import requests
import os
from dotenv import load_dotenv


load_dotenv()
# Now you can access your environment variable securely
bearer_token = os.getenv('TWITTER_BEARER_TOKEN')

# Replace 'YOUR_BEARER_TOKEN' with your actual Bearer Token
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOWisQEAAAAA0SI080kDiBsW6dkCIarxBExwt%2BU%3DaOGJ6QtXEVPESGiOLzymHuaHuIMujoN9STGbLuGqoqTlvfI3Ak'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url():
    # Replace '#crypto' with any other hashtag you're interested in
    hashtag = "%23crypto"  # URL encoded '#'
    url = f"https://api.twitter.com/2/tweets/search/recent?query={hashtag}&tweet.fields=author_id,created_at"
    return url

def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

# Set up the URL and headers
url = create_url()
headers = create_headers(bearer_token)

# Make the request
json_response = connect_to_endpoint(url, headers)

# Print the response
print(json_response)


{'data': [{'text': 'RT @Piqueenreal: Retweet retweet 🚀🚀🚀✅💯\n\nNumber of X followers : Pi Vs top ranked Cryptos\n\nX 关注者数量：Pi 与排名最高的加密货币\n\n@PiCoreTeam , #PiNetwork,…', 'author_id': '1465201393644421122', 'created_at': '2024-02-16T22:33:08.000Z', 'edit_history_tweet_ids': ['1758620555664842904'], 'id': '1758620555664842904'}, {'text': ". @NinjaBlazeApp has launched its own collection of NFTs that give you an advantage in the game 👀\n\nhttps://t.co/SBXZ1twRJS\n\nMinted 6,755/10,000 \n\nNFTs are selling out very quickly, so we don't have much time, I'll take one myself the other day 📱\n\n$INJ #Injective #NFT #Crypto https://t.co/n9DpcDxxmt", 'author_id': '1638945783138361353', 'created_at': '2024-02-16T22:33:07.000Z', 'edit_history_tweet_ids': ['1758620552602734597'], 'id': '1758620552602734597'}, {'text': 'RT @AirdropDet: 🔍 MainnetZ #Airdrop\n\n🏆 Airdrop Pool: 15,000 NETZ+ 600 USDT\n\n🔴 Start the airdrop bot: https://t.co/3apbs1T4oa\n\n🔘 Do the task…', 'author_id': '1442122794402152454',

In [14]:
tweet = json_response['data'][0]['text']
print(tweet)

RT @Piqueenreal: Retweet retweet 🚀🚀🚀✅💯

Number of X followers : Pi Vs top ranked Cryptos

X 关注者数量：Pi 与排名最高的加密货币

@PiCoreTeam , #PiNetwork,…


In [18]:
# Using Textblob to analyze sentiment example
from textblob import TextBlob

text = "Bitcoin is having a great day!"
blob = TextBlob(tweet)
sentiment = blob.sentiment

print(f"Polarity: {sentiment.polarity}, Subjectivity: {sentiment.subjectivity}")


# Polarity is a float within the range [-1.0, 1.0], where -1 means negative sentiment and 1 means positive sentiment.
# Subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.


Polarity: 0.5, Subjectivity: 0.5


In [1]:
import praw
from dotenv import load_dotenv
import os

## Reddit API

In [2]:


# Load environment variables from .env file
load_dotenv()

# Access environment variables
client_id = os.getenv('REDDIT_CLIENT_ID')
client_secret = os.getenv('REDDIT_CLIENT_SECRET')
user_agent = os.getenv('REDDIT_USER_AGENT')
username = os.getenv('REDDIT_USERNAME')
password = os.getenv('REDDIT_PASSWORD')



# Initialize PRAW with your credentials
reddit = praw.Reddit(client_id=client_id,
                     client_secret=	client_secret,
                     user_agent=user_agent,
                     username= username,
                     password= password)
print('Done')


Done


In [4]:
subreddit = reddit.subreddit("crypto")

for post in subreddit.top(limit=10):
    print(post.title, post.url)


Wikileaks latest insurance files don't match hashes https://www.reddit.com/r/crypto/comments/5cz1fz/wikileaks_latest_insurance_files_dont_match_hashes/
Crypto is not cryptocurrency http://www.cryptoisnotcryptocurrency.com
DoJ Rule 41 passively changes this Thursday, Dec 1st; Using tools like VPN or anonymizers like Tor could land you on a watch list for exploratory scanning by FBI https://www.reddit.com/r/crypto/comments/5figex/doj_rule_41_passively_changes_this_thursday_dec/
Announcing the first SHA1 collision https://security.googleblog.com/2017/02/announcing-first-sha1-collision.html
A sneaky attempt to end encryption is worming its way through Congress https://www.theverge.com/interface/2020/3/12/21174815/earn-it-act-encryption-killer-lindsay-graham-match-group
The billionaire chief executive of WhatsApp, Jan Koum, is planning to leave the company after clashing with its parent, Facebook, over the popular messaging service’s strategy and Facebook’s attempts to use its personal data